In [1]:
# In order to draw the projection graph, I need 2 points
## YTD spending
## Year budget
# This script read LIVE `transactions` table and generate Insert SQL for `projection` table
# In details, it calculate year-to-date expense amount and project (linear) end of year expense for each category
# Export SQL insert for Projection Table, save to './projections_insert.sql'
## (for each category, two row (with date equal start and end of year) specify how much expense spent at that point)

In [2]:
# Config
CATEGORY_LIST = [
    "Rent & Utilities",
    "Groceries",
    "Shopping",
    "Travel",
    "Food & Drink",
    "Entertainment",
    "Furniture & Appliances",
    "Work & Equipment",
    "Service Bills & Fees",
    "Transport",
    "Family & Personal",
    "Healthcare",
    "Other"
]
TYPE = "Expense"
YEAR_STRING = "2024"
YEAR = 2024

In [3]:
import os
import psycopg2 as ps
import pandas as pd
from dotenv import load_dotenv
load_dotenv() 
conn = ps.connect(
    host=os.environ["db_host"],
    port=os.environ["db_port"],
    dbname=os.environ["db_name"],
    user=os.environ["db_user"],
    password=os.environ["db_password"])

from datetime import date

In [4]:
# SQL query to pandas datafram
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, ps.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [5]:
# Return insert sql script as string
import re

def get_insert_query_from_df(df, dest_table):

    insert = """
    INSERT INTO {dest_table} (
        """.format(dest_table=dest_table)

    columns_string = str(list(df.columns))[1:-1]
    columns_string = re.sub(r' ', '\n        ', columns_string)
    columns_string = re.sub(r'\'', '', columns_string)

    values_string = ''

    for row in df.itertuples(index=False,name=None):
        values_string += re.sub(r'nan', 'null', str(row))
        values_string += ',\n'

    return insert + columns_string + ')\n     VALUES\n' + values_string[:-2] + ';'

In [6]:
# Query this year transactions into dataframe
def create_dataframe_for_category(category):
    column_names = ["date", "total"]
    cursor = conn.cursor()
    query = "SELECT date, SUM(abs(amount)) OVER (ORDER BY date) as total FROM transactions WHERE type IN ('" + TYPE + "') AND date_part('year', date) = '" + YEAR_STRING + "' AND category IN ('" + category + "') GROUP BY amount, date order BY date"
    df = postgresql_to_dataframe(conn, query, column_names)

    df.tail(1)
    total = df.tail(1)['total']
    

    # Calculate number of day between last entry of expense and start of this year

    start_of_year_date = date(YEAR, 1, 1)
    end_of_year_date = date(YEAR, 12, 31)
    
    # Check if total is empty (causing error when calculating day_delta)
    if isinstance(total, (int, float, complex)):
        time_delta = df.tail(1)['date'] - start_of_year_date
        print(time_delta)
        day_delta = time_delta.dt.days
        print(day_delta)

        # Calculate year end projections
        day_in_year = (end_of_year_date - start_of_year_date).days
        year_end_amount = round((total/day_delta) * day_in_year, 2).values[0]

        # Create dataframe for projections
        return pd.DataFrame({
            "date": [start_of_year_date.isoformat(), end_of_year_date.isoformat()],
            "type": [TYPE, TYPE],
            "category": [category, category],
            "amount": [0, year_end_amount]
        })
    else:
        return pd.DataFrame({
            "date": [start_of_year_date.isoformat(), end_of_year_date.isoformat()],
            "type": [TYPE, TYPE],
            "category": [category, category],
            "amount": [0, 0]
        })

In [7]:
# Create dataframe for projections
df_projections = pd.DataFrame()
for category in CATEGORY_LIST:
    df_projections = pd.concat([df_projections, create_dataframe_for_category(category)])

insert_sql = (get_insert_query_from_df(df_projections, 'projections'))
file1 = open('./projections_insert.sql', 'w')
file1.write(insert_sql)
file1.close()

total
4    1180.5
Name: total, dtype: float64
False
total
14    256.72
Name: total, dtype: float64
False
total
17    221.39
Name: total, dtype: float64
False
total
5    40.56
Name: total, dtype: float64
False
total
13    179.06
Name: total, dtype: float64
False
total
5    84.63
Name: total, dtype: float64
False
total
0    124.75
Name: total, dtype: float64
False
total
0    0.99
Name: total, dtype: float64
False
total
0    5.0
Name: total, dtype: float64
False
total
1    51.2
Name: total, dtype: float64
False
total
Series([], Name: total, dtype: object)
False
total
3    772.6
Name: total, dtype: float64
False
total
Series([], Name: total, dtype: object)
False
